# Lesson 3 - Building a RAG Agent with CrewAI

In this lesson, you will build a RAG agent with `CrewAI`. You will do that by integrating `RagTool` from `crewai_tools` with a `CrewAI` agent. `RagTool` provides a way to create and query knowledge bases from various data sources, and allows the agent to access specialized context. In this lesson, you will provide the RAG tool a pdf file containing details about insurance coverage provided by a private health insurer. By the end of the lesson, you will build an insurer agent specialized in answering queries related to health benefits. In the next lessons, you will wrap this agent in an ACP server and make it interact with other ACP agents.

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To Access <code>requirements.txt</code> and the <code>data</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

## 3.1. Import Libraries

In [1]:
import sys
!{sys.executable} -m pip install 'crewai[tools]'

In [2]:
from crewai import Crew, Task, Agent, LLM
from crewai_tools import RagTool

/Users/ytchen/Documents/experimental/ACP/env/lib/python3.10/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [3]:
import warnings
warnings.filterwarnings('ignore')

## 3.2. Define the Agent's Large Language Model

You'll now define the large language model that you will use for your CrewAI agent. `max_tokens`: maximum number of tokens the model can generate in a single response.

**Note**: If you will define this model locally, it requires that you define the API key in a **.env** file as follows:
```
# Required
OPENAI_API_KEY=sk-...

# Optional
OPENAI_API_BASE=<custom-base-url>
OPENAI_ORGANIZATION=<your-org-id>
```

In [4]:
llm = LLM(model="openai/gpt-4", max_tokens=1024)

## 3.3. Define the RAG Tool

For the RAG tool, you can define the model provider and the embedding model in a configuration Python dictionary. You can also define the details of your vector database. If you don't specify the vector database, the RagTool will use Chroma (ChromaDB) as the default vector database in local/in-memory mode.

In [5]:
config = {
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4",
        }
    },
    "embedding_model": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-ada-002"
        }
    }
}

You can then pass the `config` to the `RagTool`, and then specify the data source for which the knowledge base will be constructed. When embedding your data, the `RagTool` chunks your document into chunks and create an embedding vector for each chunk. You can specify the chunk size (`chunk_size`: number of characters) and how many characters overlap between consecutive chunks (`chunk_overlap`). You can also use the default behavior.

In [7]:
rag_tool = RagTool(config=config,  
                   chunk_size=1200,       
                   chunk_overlap=200,     
                  )
rag_tool.add("../data/gold-hospital-and-premium-extras.pdf", data_type="pdf_file")

Inserting batches in chromadb: 100%|█████████████████████████████████| 1/1 [00:01<00:00,  1.92s/it]


In [8]:
rag_tool

RagTool(name='Knowledge base', description="Tool Name: Knowledge base\nTool Arguments: {'query': {'description': None, 'type': 'str'}}\nTool Description: A knowledge base that can be used to answer questions.", env_vars=[], args_schema=<class 'abc.RagToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x104d3e050>, result_as_answer=False, max_usage_count=None, current_usage_count=0, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x10fc4df60>, summarize=False), config={'llm': {'provider': 'openai', 'config': {'model': 'gpt-4'}}, 'embedding_model': {'provider': 'openai', 'config': {'model': 'text-embedding-ada-002'}}})

## 3.4. Define the Insurance Agent

Now that you have the `rag_tool` defined, you define the CrewAI agent that can assist with insurance coverage queries.

In [9]:
insurance_agent = Agent(
    role="Senior Insurance Coverage Assistant", 
    goal="Determine whether something is covered or not",
    backstory="You are an expert insurance agent designed to assist with coverage queries",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[rag_tool], 
    max_retry_limit=5
)

## 3.5. Define the Agent Task

Let's now test the insurance agent. For that, you need to define the agent task and pass to it the query and the agent.

In [10]:
task1 = Task(
        description='What is the waiting period for rehabilitation?',
        expected_output = "A comprehensive response as to the users question",
        agent=insurance_agent
)

## 3.6. Run the Insurance Agent

To run the agent, you need to pass the agent and the task to a Crew object that you can run using the `kickoff` method.

In [11]:
crew = Crew(agents=[insurance_agent], tasks=[task1], verbose=True)
task_output = crew.kickoff()
print(task_output) 

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: bd68644b-e94c-416a-af6d-0a7e45fa1b02                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Task: What is the waiting period for rehabilitation?                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Thought: I need to find out the specific waiting period for rehabilitation coverage in the insurance.          │
│                                                                                                                 │
│  Using Tool: Knowledge base                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"waiting period for rehabilitation coverage\"}"                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│  Health Programs & Support Our Health and Hospital Substitution programs are designed to support our members    │
│  on their health journey or on their road to recovery. We have a number of programs to suit a variety of        │
│  health and recovery needs. HEALTH PROGRAMS Our health programs are designed to help you keep on top of your    │
│  health and live a healthier life. We have a range of health programs to help you manage a number of different  │
│  health conditions. The waiting period to recieve health programs is 2 months of continuous cover. HOSPITAL     │
│  SUBSTITUTION PROGRAMS* Our Hospital substitution programs allow you and/or your family members to recover in   │
│  the comfort of your own home with a range of in home hospital treatments, so you can have the choice of        │
│  treatments that suit your needs. You’ll need to have served your 2-month waiting period and have a referral    │
│  from your treating doctor to be eligible. Please note, anything that is a pre-existing condition will have a   │
│  12-month waiting period.                                                                                       │
│                                                                                                                 │
│  CLINICAL CATEGORIES WAITING PERIOD GOLD Rehabilitation 2 months 4 Hospital psychiatric services 2 months 4     │
│  Palliative care 2 months 4 Brain and nervous system 2 months 4 Eye (not cataracts) 2 months 4 Ear, nose and    │
│  throat 2 months 4 Tonsils, adenoids and grommets 2 months 4 Bone, joint and muscle 2 months 4 Joint            │
│  reconstructions 2 months 4 Kidney and bladder 2 months 4 Male reproductive system 2 months 4 Digestive system  │
│  2 months 4 Hernia and appendix 2 months 4 Gastrointestinal endoscopy 2 months 4 Gynaecology 2 months 4         │
│  Miscarriage and termination of pregnancy 2 months 4 Chemotherapy, radiotherapy and immunotherapy for cancer 2  │
│  months 4 Pain management 2 months 4 Skin 2 months 4 Breast surgery (medically necessary) 2 months 4 CLINICAL   │
│  CATEGORIES WAITING PERIOD GOLD Diabetes management (excluding insulin pumps) 2 months 4 Heart and vascular     │
│  system 2 months 4 Lung and chest 2 months 4 Blood 2 months 4 Back, neck and spine 2 months 4 Plastic and       │
│  reconstructive surgery (med...                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The waiting period for rehabilitation coverage under the insurance policy is 2 months of continuous cover.     │
│  However, please note that anything that is a pre-existing condition will have a 12-month waiting period.       │
│  There are also rehab at home programs available which include therapy for various conditions, but please be    │
│  aware that waiting periods may apply for these specific health programs.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b60d0649-9945-4e56-b890-fdb2dec8ed9f                                                                     │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: bd68644b-e94c-416a-af6d-0a7e45fa1b02                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The waiting period for rehabilitation coverage under the insurance policy is 2 months of         │
│  continuous cover. However, please note that anything that is a pre-existing condition will have a 12-month     │
│  waiting period. There are also rehab at home programs available which include therapy for various conditions,  │
│  but please be aware that waiting periods may apply for these specific health programs.                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The waiting period for rehabilitation coverage under the insurance policy is 2 months of continuous cover. However, please note that anything that is a pre-existing condition will have a 12-month waiting period. There are also rehab at home programs available which include therapy for various conditions, but please be aware that waiting periods may apply for these specific health programs.


## 3.7. Resources

- [CrewAI Agents](https://docs.crewai.com/concepts/agents)
- [CrewAI Tasks](https://docs.crewai.com/concepts/tasks)
- [CrewAI RagTool](https://docs.crewai.com/tools/ai-ml/ragtool)
- [Short course on Multi Agents with CrewAI](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>